In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
URL = "https://elcinema.com"
List_URL = "/en/index/work/country/EG"
PageURL = "?page="
Page=1
Movie_URLS = [] # list to store URLs for all the movies from year 2011 to year 2021
r = requests.get(URL+List_URL+PageURL+str(Page))
soup = BeautifulSoup(r.content, 'html5lib')  
while soup.head.title.get_text() != 'Page not found':
    print(URL+List_URL+PageURL+str(Page))
    print('Page: '+str(Page))
    print(soup.head.title.get_text())
    table = soup.find('table', attrs = {'class':"expand"})
    rows = table.find_all('tr')
    for row in rows: 
        cols = row.find_all('td')
        if len(cols)>2:
            if cols[2].text == 'Movie':
                #print(cols[1].a['href'])
                #print(cols[1].text)
                Movie_URLS.append( cols[1].a['href'])
    Page+=1
    r = requests.get(URL+List_URL+PageURL+str(Page))
    soup = BeautifulSoup(r.content, 'html5lib')
Page=1
print(Movie_URLS)

4111/', '/en/work/1000970/', '/en/work/1364683/', '/en/work/1010802/', '/en/work/1006466/', '/en/work/1010180/', '/en/work/1007240/', '/en/work/1044796/', '/en/work/1181773/', '/en/work/1005176/', '/en/work/1001995/', '/en/work/1006817/', '/en/work/1006223/', '/en/work/1001256/', '/en/work/1011377/', '/en/work/1010189/', '/en/work/1520104/', '/en/work/1000453/', '/en/work/1961105/', '/en/work/1002428/', '/en/work/1009493/', '/en/work/1001798/', '/en/work/1002905/', '/en/work/1303780/', '/en/work/1010289/', '/en/work/1000902/', '/en/work/1005861/', '/en/work/1011525/', '/en/work/1002252/', '/en/work/2012335/', '/en/work/2026735/', '/en/work/1194130/', '/en/work/2026736/', '/en/work/1005118/', '/en/work/1773176/', '/en/work/1778198/', '/en/work/1009145/', '/en/work/1006262/', '/en/work/1691366/', '/en/work/1010883/', '/en/work/1008874/', '/en/work/1010893/', '/en/work/1761897/', '/en/work/1011516/', '/en/work/1009968/', '/en/work/1004624/', '/en/work/1104468/', '/en/work/1008241/', '/en/

In [20]:
import csv
keys = ["id","name","description","duration","released?","release year","release date","revenue"]
with open('CSV/Movies.csv', 'w', newline='',encoding="utf-8")  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
keys = ["movie_id","cast_id","name","role"]
with open('CSV/MovieCast.csv', 'w', newline='',encoding="utf-8")  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
keys = ["movie_id","authority","rating"]
with open('CSV/MovieRating.csv', 'w', newline='',encoding="utf-8")  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
keys = ["movie_id","genre"]
with open('CSV/MovieGenre.csv', 'w', newline='',encoding="utf-8")  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()

In [23]:
import logging
import math
import re
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))
URL="http://elcinema.com"
try:
    df=pd.read_csv('CSV/Movies.csv')
    row_count=len(df.index)
except Exception:
    row_count=0
print("Starting from row "+str(row_count))
for w in Movie_URLS[row_count:]:
    print(w)
    MovieCast=[]
    MovieGenre=[]
    Movie = {}
    MRating={}
    Movie['id'] = re.sub("[^0-9]", "",w)  #gets ID from url
    r = s.get(URL+w)
    soup = BeautifulSoup(r.content, 'html5lib')  

    #Get name of movie
    table =  soup.find(('div'), attrs= {'class':"panel jumbo"})
    table = table.find('h1')
    table = table.find('span')
    Movie['name']=table.text.strip()
    #Get rating of movie


    table=soup.findAll('div', attrs={'class':None})
    for i in table:
        #print(i)
        try:
            e = i.find('div',attrs={'class':"row"}).find('div', attrs={'class':"columns large-10"})
            if(e is None):
                raise Exception("e is none")
            e = e.findAll('ul',attrs={'class':"list-separator list-title"})
            for l in e: 
                try: 
                    m=l.find('li',attrs={'class':None})
                    if m.text.strip()=="Censorship:":
                        #print("found censorship")
                        m=l.findAll('li',attrs={'class':None})
                        #print(len(m))
                        for n in m:
                            try:
                                o = n.find('ul')
                                o = o.findAll('li')
                                
                                MRating['movie_id']=Movie['id']
                                MRating['authority']=o[0].text.strip()
                                MRating['rating']=o[1].text.strip()
                               # print(MRating)
                                MovieRating.append(MRating)
                            except Exception:
                                pass
                        break
                except Exception:
                    pass

            break
        except Exception:
            pass

    #Get movie description
    Movie['description']=""
    try:
        if(len(soup.findAll('p'))<=1):
            raise Exception("No description")
        table = soup.find('p')
        Movie['description']=table.text.strip()
        Movie['description'] = re.sub("...Read more","",Movie['description'])
    except Exception:
        pass
    #Get movie duration
    try:
        table = soup.find('div', attrs={'class': "intro-box"})
        table = table.find('div', attrs = {'class':"row"})  
        table= table.find('div', attrs = {'class':"columns small-12 medium-9 large-9"})
        table = table.find('div', attrs = {'class':"row"}) 
        table= table.find('div', attrs = {'class':"columns small-9 large-10"})
        table= table.find('ul', attrs = {'class':"list-separator"})
        Movie['duration'] = table.findAll('li')[2].text.strip()
        Movie['duration'] = re.sub("[^0-9]", "",Movie['duration'])
    except Exception: 
        Movie['duration']="0"
    try:
        table = soup.find('div', attrs={'class': "intro-box"})
        table = table.find('div', attrs = {'class':"row"})  
        table= table.find('div', attrs = {'class':"columns small-12 medium-9 large-9"})
        table = table.find('div', attrs = {'class':"row"}) 
        table= table.find('div', attrs = {'class':"columns small-9 large-10"})
        table= table.find('ul', attrs = {'class':"list-separator"})
        table = table.findAll('li')
        Movie['released?']="No"
        for t in table:
            try:
                print(t)
                if ("Released" in t.text.strip()):
                    Movie['released?']="Yes"
                    print("YES")
                    break
            except Exception:
                pass
    except Exception: 
        pass
    #Get movie release date
    Movie['release year']=''
    Movie['release date']=''
    try:
        table =  soup.find(('div'), attrs= {'class':"panel jumbo"})
        table = table.find('h1')
        table = table.findAll('span')[1]
        Movie['release year']=table.text.strip()
        Movie['release year'] = re.sub("[^0-9]", "",Movie['release year'])
    except Exception:
        pass  
    try:
        table = soup.find('div', attrs={'class': "intro-box"})
        table = table.find('div', attrs = {'class':"row"})  
        table= table.find('div', attrs = {'class':"columns small-12 medium-9 large-9"})
        table = table.find('div', attrs = {'class':"row"}) 
        table= table.find('div', attrs = {'class':"columns small-9 large-10"})
        table = table.findAll('ul',attrs={'class':"list-separator"})
       # print(len(table))
        for i in table:
            try:
                #print("in forLoop")
                #print(i)
                m=i.find('li').find('ul').find('li')
                #print(m.text.strip())
                if "Release Date:" in m.text.strip():
                    m=m.findAll('a')
                    Movie['release date'] = ""
                    for e in m[:2]:
                        Movie['release date'] += e.text.strip()+" "
                    Movie['release date']=Movie['release date'][:-1]
                  #  print(Movie['release date'])

                    break
            except Exception as exStr:
                #print("Fail")
                pass

    except Exception: 
        Movie['release date']="01 Jan 0000"

    #Gets Revenue
    Movie['revenue']=""
    try: 
        table=soup.findAll('div',attrs = {'class' : None})[4]
        table=table.find('div')
        table=table.find('div',attrs={'class':"columns large-10"})
        bo = table.findAll('ul', attrs={ 'class':"list-separator list-title"})[-1]
        if(bo.find('li').text.strip()=='Box Office:'):
            Movie['revenue']=bo.findAll('li')[1].text.strip()
        Movie['revenue'] = re.sub("[^0-9]", "",Movie['revenue'])
    except Exception:
        pass

    ##gets movie genres##
    table=soup.findAll('div', attrs={'class':None})
    #print(len(table))
    for i in table:
        #print(i)
        try:
            e = i.find('div',attrs={'class':"row"}).find('div', attrs={'class':"columns large-10"})
            if(e is None):
                #print("NONE")
                raise Exception("e is none")
            #print(e)
            e = e.find('ul',attrs={'id':"jump-here-genre"})
            if(e is None):
               # print("NONE")
                raise Exception("e is none")
            #print(e)
            try: 
                e=e.findAll('li',attrs={'class':None})
                if e[0].text.strip()=="Genre:":
                    #print("found Genre")
                   # e=e.findAll('li',attrs={'class':None})
                    #print(e)
                    #print(len(m))
                    for n in e:
                        try:
                            o = n.find('a')
                            #print(o.text.strip())
                            MovieGenreEntry={}                        
                            MovieGenreEntry['movie_id']=Movie['id']
                            MovieGenreEntry['genre']=o.text.strip()
                            MovieGenre.append(MovieGenreEntry)
                        except Exception:
                            pass
                    break
            except Exception:
                pass
        except Exception:
            pass
    #Gets cast for movie
    r = s.get(URL+w+"cast")
    print(URL+w+"cast")
    soup = BeautifulSoup(r.content, 'html5lib')
    try:
        table = soup.find('hr', attrs = {'class':"split"}).find_next_sibling('div')
        role_table=table.find('div',attrs={'class':'columns large-2'})
        while(role_table) :
            role=role_table.find('h3', attrs = {'class':"section-title"}).text.strip()   
            role=re.sub("[^a-zA-Z]","",role)
            #print(role)
            role_entries=role_table.find_next_sibling('div',attrs={'class':"columns large-10"}).find('ul')
            role_entries=role_entries.findAll('li',attrs={'class':None})
           # print(role_entries[0])
            for i in role_entries:
                try:
                    PersonEntry=i.find('div').find('ul',attrs={'class':"description"}).find('li')
                    MovieCastEntry={}                        
                    MovieCastEntry['movie_id']=Movie['id']
                    MovieCastEntry['cast_id']=PersonEntry.a['href']
                    MovieCastEntry['cast_id']=re.sub("[^0-9]","",MovieCastEntry['cast_id'])
                    MovieCastEntry['name']=PersonEntry.a.text.strip()
                    MovieCastEntry['role']=role
                    MovieCast.append(MovieCastEntry)
                except Exception:
                    pass
                            #print(MovieCastEntry)
            table=table.find_next_sibling('div',attrs={'class':'row'})
            if(table):
                role_table=table.find('div',attrs={'class':'columns large-2'})
            else:
                role_table=None
    except Exception:
        pass
    keys = Movie.keys()
    with open('CSV/Movies.csv', 'a', newline='',encoding="utf-8")  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writerow(Movie)
    if len(MovieCast)>0:
        keys = MovieCast[0].keys()
        with open('CSV/MovieCast.csv', 'a', newline='',encoding="utf-8")  as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writerows(MovieCast)
    keys = MRating.keys()
    with open('CSV/MovieRating.csv', 'a', newline='',encoding="utf-8")  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writerow(MRating)
    if len(MovieGenre)>0:
        keys = MovieGenre[0].keys()
        with open('CSV/MovieGenre.csv', 'a', newline='',encoding="utf-8")  as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writerows(MovieGenre)


work/2026438/
DEBUG:urllib3.connectionpool:http://elcinema.com:80 "GET /en/work/2026438/ HTTP/1.1" 301 178
DEBUG:urllib3.connectionpool:https://elcinema.com:443 "GET /en/work/2026438/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://elcinema.com:80 "GET /en/work/2026438/cast HTTP/1.1" 301 178
<li>Movie</li>
<li><a href="/en/index/work/country/eg">Egypt</a></li>
<li>Released</li>
YES
DEBUG:urllib3.connectionpool:https://elcinema.com:443 "GET /en/work/2026438/cast HTTP/1.1" 200 None
http://elcinema.com/en/work/2026438/cast
/en/work/2025679/
DEBUG:urllib3.connectionpool:http://elcinema.com:80 "GET /en/work/2025679/ HTTP/1.1" 301 178
DEBUG:urllib3.connectionpool:https://elcinema.com:443 "GET /en/work/2025679/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:http://elcinema.com:80 "GET /en/work/2025679/cast HTTP/1.1" 301 178
<li>Movie</li>
<li><a href="/en/index/work/country/eg">Egypt</a></li>
<li>51 minutes</li>
<li>Released</li>
YES
DEBUG:urllib3.connectionpool:https://elcinema.com:4